In [ ]:
import os
import pickle

import numpy as np
import sophus as sp
import polymetis
from torchcontrol.transform import Rotation as R

import fairotag as frt
from realsense_wrapper import RealsenseAPI

In [ ]:
NUM_CAMERAS = 3
NUM_SAMPLES = 20

TABLE_MARKER = (0, 0.1)
EE_MARKER = (8, 0.04)

DATA_DIR = "data/2_hand_eye_calibration"

Instantiate camera modules

In [ ]:
with open("data/realsense_intrinsics.pkl", 'rb') as f:
    intrinsics = pickle.load(f)

cameras = []
for _ in range(NUM_CAMERAS):
    camera = frt.CameraModule()
    camera.set_intrinsics(intrinsics)
    cameras.append(camera)

Instantiate scene

In [ ]:
scene = frt.Scene()

scene.add_frame("ee")
for i, camera in enumerate(cameras):
    scene.add_camera(f"camera_{i + 1}")

Register markers

In [ ]:
# Register markers in cameras
for camera in cameras:
    for marker in [TABLE_MARKER, EE_MARKER]:
        camera.register_marker_size(marker[0], marker[1])

# Register markers in scene
scene.add_marker(TABLE_MARKER[0], length=TABLE_MARKER[1])
scene.add_marker(EE_MARKER[0], length=EE_MARKER[1], frame="ee")

Load calibration data

In [ ]:
# Load EE pose data
ee_pose_data = numpy.load(os.path.join(DATA_DIR, "ee_poses.npz"))

ee_poses = []
for i in range(NUM_SAMPLES):
    ee_pos = ee_pose_data["pos"][i, :]
    ee_quat = ee_pose_data["quat"][i, :]
    ee_pose = frt.utils.xyz_quat_to_se3(ee_pos, ee_quat)
    ee_poses.append(ee_pose)

# Load image data
marker_observations = []
for i in range(NUM_SAMPLES):
    imgs = []
    for j in range(NUM_CAMERAS):
        filename = os.path.join(DATA_DIR, f"camera{j}_{i}.jpg")
        img = cv2.imread(filename)
        imgs.append(img)

    marker_obs = [camera.detected_markers(img) for camera, img in zip(cameras, imgs)]
    marker_observations.append(marker_obs)

Optimize & visualize results

In [ ]:
scene.calibrate_extrinsics(
    detected_markers_ls=marker_observations,
    frame_transforms_ls=[("world", "ee", ee_pose) for ee_pose in ee_poses],
    verbosity=1,
)

scene.visualize()